<a href="https://colab.research.google.com/github/thasliya0404/internship/blob/main/q10_Topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 3.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
import hdbscan
from sklearn.feature_extraction.text import CountVectorizer


/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [ ]:
df = pd.read_csv("/content/mergeddata (1).csv")

In [ ]:
df_q10 = df[df['QUESTION_ID'] == 10]
df_q10.shape

(19707, 14)

In [ ]:
df_q10 = df_q10[
    (df_q10["REMARKS"].notna()) &
    (df_q10["REMARKS"].astype(str).str.strip() != "")
].copy()
# Create raw_text explicitly
df_q10["raw_text"] = df_q10["REMARKS"].astype(str)



NON_INFORMATIVE_PATTERNS = [
    r"^nothing$",
    r"^nothing much$",
    r"^nothing in particular$",
    r"^no comment(s)?$",
    r"^null$",
    r"^n/?a$",
    r"^na$",
    r"^everything$",
    r"^everything is (good|fine|ok|alright)(.*)?$",
    r"^overall (good|fine|ok|excellent|nice)(.*)?$",
    r"^it was excellent(.*)?$",
    r"^excellent(.*)?$",
    r"^good$",
    r"^very good(.*)?$",
    r"^wonderful(.*)?$",
    r"^outstanding(.*)?$",
    r"^like everything(.*)?$",
    r"^all (thing|things)(.*)?$",
    r"^nice sir$",
    r"^good sir$",
    r"^all$",
    r"^none$",
    r"^nthg$",
    r"^nan$",
    r"^nil$",
    r"good class$",
    r"nice class$",
    r"its ok$",
    r"nothimg$",
    r"gud$",
    r"non$",
    r"nop$",
    r"yes$",
    r"nul$",
    r"nuce$",
    r"noo$",
    r"really$"


]

def is_non_informative(text):
    text = text.lower().strip()
    for pattern in NON_INFORMATIVE_PATTERNS:
        if re.fullmatch(pattern, text):
            return True
    return False
df_q10["is_informative"] = ~df_q10["raw_text"].apply(is_non_informative)

TEACHING_CONTEXT = [
    "class", "lecture", "teaching", "explain", "interactive",
    "boring", "interesting", "pace", "topic", "sir", "teacher","miss"
]
GENERIC_WORDS = {
    "good", "nice", "excellent", "awesome",
    "great", "best", "perfect",
    "sir", "madam", "teacher", "faculty","class","miss"
}
GENERIC_PATTERN = r"\b(" + "|".join(GENERIC_WORDS) + r")\b"

def normalize_section_session(text):
    text = text.lower()
    if "section" in text:
        for w in TEACHING_CONTEXT:
            if w in text:
                text = re.sub(r"\bsection\b", "session", text)
                break
    return text


def preprocess_for_bertopic(text):

    if pd.isna(text):
        return None

    text = text.lower().strip()

    if is_non_informative(text):
        return None
    # Remove generic words (string-level)
    text = re.sub(GENERIC_PATTERN, " ", text)

    text = normalize_section_session(text)

    # Remove numbers & symbols
    text = re.sub(r"[^a-z\s]", " ", text)

    text = re.sub(r"\s+", " ", text)
    # Remove very short feedback after cleaning
    if len(text) < 3:
        return None

    return text
df_q10["cleaned"] = df_q10["REMARKS"].apply(preprocess_for_bertopic)
df_q10 = df_q10[df_q10["cleaned"].notna()]
documents = df_q10["cleaned"].tolist()


For saving the dataset

In [ ]:
len(documents)

16407

In [ ]:
import hdbscan
embedding_model = SentenceTransformer("all-mpnet-base-v2")

umap_model = UMAP(
    n_neighbors=20,
    n_components=5,
    min_dist=0.1,          # ⬆️ allows better grouping
    metric="cosine",
    random_state=42
)


hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=40,   # ⬆️ merge similar ideas
    min_samples=5,         # ⬇️ less strict
    metric="euclidean",
    prediction_data=True
)

vectorizer_model = CountVectorizer(
    ngram_range=(1, 2),
    stop_words="english",
    min_df=5               # 🔥 removes rare noise words
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    calculate_probabilities=True,
    verbose=True
)

topics, probs = topic_model.fit_transform(documents)
#df_q10["topic"] = topics


2025-12-28 03:42:03,009 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/513 [00:00<?, ?it/s]

2025-12-28 03:49:32,164 - BERTopic - Embedding - Completed ✓
2025-12-28 03:49:32,165 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-28 03:50:43,370 - BERTopic - Dimensionality - Completed ✓
2025-12-28 03:50:43,373 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-28 03:51:09,292 - BERTopic - Cluster - Completed ✓
2025-12-28 03:51:09,306 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-28 03:51:09,461 - BERTopic - Representation - Completed ✓


In [ ]:
df_q10["topic"].nunique()

121

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 19.8 MB/s eta 0:00:00


In [ ]:
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

# 1️⃣ Tokenize documents
texts = [doc.split() for doc in documents]

# 2️⃣ Create dictionary
dictionary = Dictionary(texts)

# 3️⃣ Extract topics safely
topics_for_coherence = []

for topic_id, topic_words in topic_model.get_topics().items():
    if topic_id == -1:
        continue  # skip outliers

    valid_words = []
    for word, _ in topic_words:
        # split bigrams and keep only words in dictionary
        for w in word.split():
            if w in dictionary.token2id:
                valid_words.append(w)

    # only keep topics with enough words
    if len(valid_words) >= 2:
        topics_for_coherence.append(valid_words)

# 4️⃣ Compute coherence
coherence_model = CoherenceModel(
    topics=topics_for_coherence,
    texts=texts,
    dictionary=dictionary,
    coherence="c_v"
)

coherence_score = coherence_model.get_coherence()
print("Coherence Score:", coherence_score)


Coherence Score: 0.4224789895621841


In [ ]:
def topic_diversity(topics, top_k=10):
    words = []
    for topic in topics:
        words.extend(topic[:top_k])
    return len(set(words)) / len(words)

topics_words = [
    [word for word, _ in topic_model.get_topic(t)]
    for t in topic_model.get_topics()
    if t != -1
]

print("Topic Diversity:", topic_diversity(topics_words))


Topic Diversity: 0.36833333333333335


In [ ]:
outlier_ratio = (df_q10["topic"] == -1).mean()
print("Outlier Ratio:", outlier_ratio)

Outlier Ratio: 0.18034985067349302


In [ ]:
topic_model.reduce_topics(
    documents,
    nr_topics=40
)


2025-12-28 03:51:21,946 - BERTopic - Topic reduction - Reducing number of topics
2025-12-28 03:51:22,006 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-28 03:51:22,320 - BERTopic - Representation - Completed ✓
2025-12-28 03:51:22,329 - BERTopic - Topic reduction - Reduced number of topics from 121 to 40


In [ ]:
topic_model.get_topic_info().shape


(40, 5)

In [ ]:

# 1️⃣ Tokenize documents
texts = [doc.split() for doc in documents]

# 2️⃣ Create dictionary
dictionary = Dictionary(texts)

# 3️⃣ Extract topics safely
topics_for_coherence = []

for topic_id, topic_words in topic_model.get_topics().items():
    if topic_id == -1:
        continue  # skip outliers

    valid_words = []
    for word, _ in topic_words:
        # split bigrams and keep only words in dictionary
        for w in word.split():
            if w in dictionary.token2id:
                valid_words.append(w)

    # only keep topics with enough words
    if len(valid_words) >= 2:
        topics_for_coherence.append(valid_words)

# 4️⃣ Compute coherence
coherence_model = CoherenceModel(
    topics=topics_for_coherence,
    texts=texts,
    dictionary=dictionary,
    coherence="c_v"
)

coherence_score = coherence_model.get_coherence()
print("Coherence Score:", coherence_score)


Coherence Score: 0.4113247179743985


In [ ]:
def topic_diversity(topics, top_k=10):
    words = []
    for topic in topics:
        words.extend(topic[:top_k])
    return len(set(words)) / len(words)

topics_words = [
    [word for word, _ in topic_model.get_topic(t)]
    for t in topic_model.get_topics()
    if t != -1
]

print("Topic Diversity:", topic_diversity(topics_words))


Topic Diversity: 0.5948717948717949


In [ ]:
outlier_ratio = (df_q10["topic"] == -1).mean()
print("Outlier Ratio:", outlier_ratio)

Outlier Ratio: 0.18034985067349302


In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.head(45)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2959,-1_clear_content_learning_delivery,"[clear, content, learning, delivery, interesti...","[content delivery, content delivery, content d..."
1,0,1742,0_teaching_way teaching_way_students,"[teaching, way teaching, way, students, instru...","[way of teaching, way of teaching, the way tea..."
2,1,1728,1_hands_session_training_practical session,"[hands, session, training, practical session, ...","[hands on training, hands on training, hands o..."
3,2,1653,2_interactive_interaction_interactive session_...,"[interactive, interaction, interactive session...","[interactive session, interactive session, int..."
4,3,1519,3_explanation_easy understand_understand_easy,"[explanation, easy understand, understand, eas...","[easy to understand, easy to understand, easy ..."
5,4,645,4_presentation_lecture_way_style,"[presentation, lecture, way, style, simple, sk...","[ presentation, presentation, presentation]"
6,5,478,5_communication_friendly_patient_approach,"[communication, friendly, patient, approach, i...","[communication, communication, friendly inter..."
7,6,409,6_topics_topic_new_new things,"[topics, topic, new, new things, things, learn...","[topics, recap of older topics, new topic for me]"
8,7,407,7_comfortable_active_free_amazing,"[comfortable, active, free, amazing, liked, ca...","[comfortable, feels free and comfortable, comf..."
9,8,393,8_understandable___,"[understandable, , , , , , , , , ]","[understandable, understandable, understandable]"


In [ ]:
for topic in topic_info["Topic"]:
    if topic != -1:
        print(topic, topic_model.get_topic(topic))

0 [('teaching', np.float64(0.13821724510965655)), ('way teaching', np.float64(0.08612106601224881)), ('way', np.float64(0.06994922224408773)), ('students', np.float64(0.06405211913957108)), ('instructor', np.float64(0.052548472204697216)), ('method', np.float64(0.04669445422936569)), ('style', np.float64(0.04547686767562791)), ('taught', np.float64(0.030390587651854678)), ('interaction', np.float64(0.022926515246015907)), ('skill', np.float64(0.02249267277222603))]
1 [('hands', np.float64(0.0801185027536978)), ('session', np.float64(0.07060706592701793)), ('training', np.float64(0.06557907596297617)), ('practical session', np.float64(0.056972004061364545)), ('practical', np.float64(0.044875445238658114)), ('sessions', np.float64(0.043613164405666455)), ('practice', np.float64(0.03861469639779685)), ('experience', np.float64(0.03841786130554688)), ('hands experience', np.float64(0.037215142371381654)), ('practice session', np.float64(0.02496536393169843))]
2 [('interactive', np.float64(

In [ ]:
topics, probs = topic_model.fit_transform(documents)
df_q10["topic"] = topics


2025-12-28 04:17:05,561 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/513 [00:00<?, ?it/s]

2025-12-28 04:23:58,320 - BERTopic - Embedding - Completed ✓
2025-12-28 04:23:58,321 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-28 04:24:50,897 - BERTopic - Dimensionality - Completed ✓
2025-12-28 04:24:50,899 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-28 04:25:19,615 - BERTopic - Cluster - Completed ✓
2025-12-28 04:25:19,616 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-12-28 04:25:19,895 - BERTopic - Representation - Completed ✓
2025-12-28 04:25:19,897 - BERTopic - Topic reduction - Reducing number of topics
2025-12-28 04:25:19,987 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-28 04:25:20,192 - BERTopic - Representation - Completed ✓
2025-12-28 04:25:20,199 - BERTopic - Topic reduction - Reduced number of topics from 121 to 40


In [ ]:
topic_model.save(
    "bertopic_q10_model_min",
    serialization="pickle"
)


2025-12-28 04:26:17,224 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

embedding_model = SentenceTransformer("all-mpnet-base-v2")

topic_model = BERTopic.load(
    "bertopic_q10_model_min",
    embedding_model=embedding_model
)


In [ ]:
topic_model.get_topic_info().shape

(40, 5)

In [ ]:
topic_model.save(
    "bertopic_q10_min",
    serialization="safetensors",
    save_embedding_model=True
)


In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==5])

['Highly efficient and have a clear knowledge about the topic he is handling',
 'It must confidence to speaking. ',
 'Kind and calm',
 'Friendly and Interactive',
 'Interactive and friendly ',
 'friendly',
 'friendly',
 'Communication',
 'Friendly ',
 'Good communication',
 'Friendly atmosphere ',
 'Friendly atmosphere ',
 'Friendly ',
 'Friendly',
 'Friendly interaction',
 'Friendly ',
 'Communication ',
 'could communicate easily',
 'Friendly and kind',
 'Every clear, friendly, approachable and have clear knowledge of the subject.',
 'Clear and patient',
 'Traiers are so kind',
 'Good communication',
 'Good communication',
 'Good communication',
 'Gopd communication',
 'Very kind ',
 'Sirs charisma',
 'Have knowledge and very friendly and approachable for doubts',
 'Nice communication ',
 'Friendly',
 'Friendly ',
 'Communication ',
 'Good communication',
 'Friendly and knowledgeable person ',
 'Friendly behaviour ',
 'Friendly',
 'Communication ',
 'Good communication ',
 'Friendly'

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==0])

['His teaching',
 'The way the instructors went around the class checking if everyone understood the topics',
 'Way of teaching ',
 'More intresting as instructor includes more from outside area while he talks ',
 'Easily understandable and supportive instructors',
 'About the interaction and way of teaching',
 'Way of teaching is good ',
 'The way of teaching was very good ',
 'Good teaching ',
 'Instructor has good knowledge ',
 'Effective and understanding and motivating teaching',
 'Teaching and image generate',
 'The way the instructor made sure everyone was following ',
 'active interaction between teacher and student',
 'Classes start from the basics ',
 'Effective teaching and proper guiding ',
 'Teaching method ,interaction  with students  Excellent and helps in decoding .',
 'The mentor and interaction ',
 'Qualified and Supportive instructor ',
 'thorough delivery of topics and instructors interaction with the students',
 'The practice given as project',
 'Proper teaching of

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==1])

['Practical practise',
 'The trainers are very friendly and help us to deal with real time projects,situation and examples',
 'Good Session',
 'Good Session ',
 'Good Session',
 'Trainer is knowledgeable ',
 'The trainer is brilliant ',
 'Trainer have good knowledge and share d',
 'Remya maam was very interactive and helpful for the entire time. Sessions was very well paced and effective. ',
 'Class was so interesting. All the session helped me to improve my knowledge and increase confidence in the field. Maam was so supportive. ',
 'training',
 'Friendly training ',
 'Hands on experience along with learning ',
 'Effective training ',
 'Very helpful and friendly trainer',
 'The overall MERN stack sessions was splendid. Learned many things. ',
 'Copilot experience ',
 'Trainers gives us Munch at last day',
 'Nice training ',
 'Quick an Easy training.',
 'Break ',
 'The training session was very good. The instructor explained topics with the help of scenarios.',
 'Practice section ',
 'P

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==2])

['Interaction ',
 'Interaction ',
 'Interaction ',
 'Interactive session',
 'Interactive ',
 'Interaction of faculty ',
 'Interactive Session',
 'Was so interactive and active',
 'Interactive ',
 'Intractive sessions',
 'Interactive ',
 'Interaction',
 'Interaction',
 'Interaction ',
 'Interactive ',
 'How interative it is',
 'Interactive ',
 'Interactive session',
 'Interaction ',
 'Interactive ',
 'How interactive it is',
 'Interaction',
 'Interactive ',
 'Interactive ',
 'Interaction ',
 'Interaction',
 'Interactive ',
 'Interactive',
 'Interactive',
 'Interaction ',
 'Interactive section',
 'Good and Interactive ',
 'Interactive ',
 'Interactive session',
 'Audible',
 'Interactive sessions ',
 'Clarifications and Interactivity of the class',
 'Interactive',
 'Interactive ',
 'Interactive ',
 'The interactive approach of the session.',
 'Interactive session',
 'Very Interactive ',
 'Interactive session',
 'Interactive session',
 'Interactive session ',
 'Interactive section ',
 'Int

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==3])

['Very nice class easyly understandable',
 'Very much intractive and informative class.highly understandble',
 'Understanding',
 'They are explained well',
 'They explain it very well',
 'It was explained in simple words and briefly ',
 'Explains every thing in  detail',
 'Explanation ',
 'Explanation ',
 'Explain formulas in simple way',
 'Explained very well',
 'Well explained ',
 'Easily Understandable ',
 'Good way',
 'Easy to understand ',
 'Easy to understand ',
 'Good explanation ',
 'Good explanation ',
 'Interactive and ready to clear doubts.',
 'Easily Understandable ',
 'His explanations',
 'Informative, helpful',
 "It's very helpful . Helped to learn something new ",
 'Explained every thing in detail  and also explained personal doubts',
 'Easy to understand ',
 'Easy to understand ',
 'Takes time for everyone to understand ',
 'Well explained',
 'Easy to understand ',
 'Very easy to learn',
 'Explanation is good',
 'good explanation, make concept clear and follow a systema

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==4])

['Easy and understanding lecture ',
 'Presentation',
 'Presentation was really good.',
 'Presenting the information to us efficiently',
 'Lecturer',
 'Presentation of class',
 'Presentation',
 'Presentation ',
 'Presentation ',
 'Presentation ',
 'Simple way of presentation',
 'Presentation ',
 'Presentation ',
 'Presentation',
 'Presentation ',
 'Good presentation',
 'Good presentation ',
 'Presentation ',
 'Lecture',
 'Good presentation ',
 'Presentation ',
 'Good presentation ',
 'Presentations ',
 'Good presentation ',
 'Presentation ',
 'Presentation ',
 'Good presentation ',
 'The way of presentation was clear and understanding ',
 'Good presentation ',
 'Presentation ',
 'Good presentation',
 'Good presentation ',
 'Lecturing ',
 'Good presentation ',
 'Presentation ',
 'Presentation ',
 'His presentation ',
 'Presentation style revising model',
 'Nice presentation ',
 'Presentation ',
 'Presentation ',
 'Presentation ',
 'Presentation ',
 'Presentation ',
 'Presentation ',
 'Pr

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==6])

['Good knowledge ',
 'Interesting cls, learning new things ',
 'Gained practical knowledge.',
 'Learned new things ',
 'New knowledge ',
 'Interactive and clarity in topics',
 'Topics were explained the most simplest and easiest way thus making it easy for me to grasp the topics.',
 'Ensuring that every topic is clear',
 'Topic',
 'Topics delivered repeatedly for understanding ',
 'Delivers every topic clearly and ensures everyone understands it',
 'got to learn new things',
 'Learned new things',
 'all the topics',
 'An opportuniy to have an internship section in ICT accademy',
 'Explore new and easy ways to do things ',
 'Explore new and easy way to do things',
 'Interesting new topic ',
 'Learned new stuff ',
 'Detailed and repeated explanation of topics',
 'Quality in delivery of topics',
 'More knowledge gaining ',
 'New topic for me',
 'Smooth and clear topic delivery ',
 'Understand the topic clearly',
 'Preparing notes from internet for better understanding.',
 'Give time to re

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==7])

['Very nice class',
 'Calm',
 'Works',
 'Amazing',
 'Energetic ',
 'Relaxing',
 'nhgthgfh',
 'Paatshala',
 'It was quite nice class',
 'Tryhackme',
 'Not at all stressful',
 'calm and clear',
 'Sir goated. Come Valo',
 'Stress free',
 'Interactions and stress-free ',
 'amazing',
 'The class is beautiful ',
 'not much stressful',
 'Stress free session',
 'Evrythin',
 'Adipoli anu ',
 'Stress free ',
 'Free ',
 'Very active and cooperative',
 'Its was a wonderful week ',
 'Class was very nice ',
 'extremely goo',
 'Amazing ',
 'It is not stressful',
 'kozhappila ellam set aayirunu',
 'i didnt have anything to like',
 'padas',
 'Enjoyable',
 'It was very good',
 'good cls',
 'its goood',
 'As usual',
 'It was very good',
 'Exacalent cls',
 'Better than yesterday',
 'kollam poli sanam ',
 'Lovely class',
 'Verry good',
 'Much more energetic class ',
 'It was abpve average',
 'chill',
 'Elam nalth ahrnu',
 'kozhappam illa adipoli ayirunnu',
 'MERN ',
 'The class in malayalam is comfortable'

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==8])

['Understandable ',
 'Understandable class',
 'Understandable ',
 'Understandable ',
 'Understandable ',
 'Understandable',
 'Understandable ',
 'Understandable ',
 'Understandable',
 'Understandable ',
 'Understandable ',
 'Understandable',
 'Understandable',
 'Understandable ',
 'Understandable ',
 'Understandable',
 'Explainable',
 'Understandable',
 'Understandable ',
 'Understandable',
 'Understandable',
 'Understandable',
 'Understandable ',
 'Understandable',
 'Understandable ',
 'Understandable',
 'Understandable ',
 'Understandable',
 'Understandable ',
 'Understandable',
 'Understandable ',
 'Understandable ',
 'Understandable ',
 'Understandable ',
 'Understandable ',
 'Understandable ',
 'understandable',
 'Understandable',
 'Understandable',
 'Understandable',
 'Understandable',
 'understandable',
 'Understandable ',
 'Understandable',
 'Good class understandable ',
 'Understandable',
 'Understandable ',
 'Understandable',
 'understandable',
 'understandable',
 'understand

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==9])

['Patiently correcting the error ',
 'Doubt clearing sections',
 'Clearing doubts efficiently ',
 'Clearing doubts',
 'Doubt clearing was so great',
 'doubt clearing',
 'clear content',
 'Doubt Clearance Session ',
 'Clarity of the class',
 'clear',
 'Showing examples in class itself and fixing all errors ',
 'helps with everyones errors',
 'doubt clearing',
 'Kind.  Cleared all doubts ',
 'Clearing doubts ',
 'Clear',
 'Cleared my doubts',
 'Error clearing ',
 'Clear',
 'Refresher area',
 'Doubt clearing',
 'prompt response and doubt clearing',
 'Understanding and doubt clearing',
 'Doubt clearing ',
 'Logical ',
 'Clarity ',
 'Doubt clearing ',
 'Very friendly, Understanding, doubt clearing is very effective',
 'Cleared all doubts ',
 'Doubt clearing ',
 'go clarity ',
 'Doubt clearing ',
 'Timely support in correction',
 'Doubt clearing and understandable',
 'Clarity ',
 'Clarity',
 ' cleared all the doubts asked',
 'repeat the section when anyone has doubt',
 'Helped in correcting 

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==10])

['activities',
 'Application sessions',
 'Activity',
 'Activities',
 'Activity ',
 'Activities ',
 'Activity ',
 'Activities ',
 'Activities ',
 'Activity ',
 'Activities ',
 'Activities ',
 'Activity',
 'activity',
 'Activities ',
 'Bingo session ',
 'Bingo session',
 'bingo',
 'bingo',
 'Activities ',
 'Activity',
 'Activities ',
 'Events',
 'Activities ',
 'Activities ',
 'Activities ',
 'events',
 'Activities ',
 'Activities ',
 'Activity ',
 'Activites',
 'Activities ',
 'gaming section',
 'LESSONS ARE DONE WITH GAMES ',
 'Activity ',
 'Activities ',
 'Activity',
 'gaming',
 'Activity ',
 'Activity ',
 'Activities',
 'Activities ',
 'Activity',
 'Activities ',
 'Activities ',
 'Activities ',
 'Activities',
 'Activity ',
 'Activity ',
 'Activities',
 'Activities ',
 'Gaming part',
 'Activities ',
 'Activities',
 'Activities ',
 'Activities',
 'Activity that done in this section ',
 'The games played as a part of learning',
 ' activity',
 'the activity',
 'activities',
 'THE ACTIVIT

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==11])

['The quiz and assingnment given at last ',
 'Quiz',
 'Q&A',
 'Quiz',
 'Project of problems and solutions ',
 'Quiz',
 'Example problems ',
 'Got practical questions to improve the understanding on topics',
 'Queries that practiced ',
 'Quiz included ',
 'Quiz',
 'Quiz',
 'Quiz',
 'Quiz',
 'Quiz time',
 'Quiz ',
 'quiz competition',
 'Quiz .',
 'Quiz',
 'Quiz',
 'quiz',
 'Example questions ',
 'Live Quizes',
 'Quiz',
 'Quiz and other activities ',
 'Quiz',
 'Quiz',
 'Quiz ',
 'Quiz',
 'Quiz',
 'problem solving via libreries',
 'quiz',
 'Quizzes section ',
 'good experience',
 'Quiz section',
 'question answering like quiz',
 'Today i liked the quiz session ',
 'Quiz',
 'Quizz section',
 'Quiz.....all thing',
 'quiz',
 'Practice questions ',
 'quiz',
 'Practice problems',
 'Problem solving practice ',
 'Questions ',
 'quiz,class',
 'Live Quiz ',
 'Quiz',
 'Practicing questions',
 ' Practice Questions',
 'giving challenging questions ',
 'Practice problem',
 'Problem solving',
 'More pra

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==12])

['Informative ',
 'Informative ',
 'It was useful',
 'Class work ',
 'Class work',
 'Class work',
 'Class work',
 'Class work',
 'Class work',
 'Very informative ',
 'Class work',
 'Class work',
 'Very informative ',
 'Informative ',
 'Proactive ',
 'Informative ',
 'Informative',
 'Effective ',
 'Effective',
 'Working ',
 'Informative ',
 'Effective ',
 'Effective',
 'Effective ',
 'Informative',
 'Informative ',
 'Very coperative',
 'Applicable',
 'Efficiency ',
 'Informative',
 'Very Useful',
 'Useful',
 'Very informative ',
 'Useful ',
 'Very informative',
 'Informative ',
 'INFORMATIVE',
 'It seemed effective',
 'Informative ',
 'informative',
 'Effective and flexibility',
 'Informative ',
 'informative ',
 'Work',
 'Useful class,highly recommendable ',
 'Work given',
 'Very useful ',
 'Effective ',
 'Effective ',
 'impactful',
 'work',
 'Class work',
 'useful',
 'Informative',
 'Informative ',
 'useful',
 'Very innovative and interesting ',
 'Very usefull and innovative class',
 

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==13])

['Practical method ',
 'Practical class',
 'Interactive and practical method ',
 'Practicals',
 'practical',
 'Practical ',
 'Got practical knowledge ',
 'Practical ',
 'Theory ',
 'Practical ',
 'Practical ',
 'PRACTICAL KNOWLEDGE',
 'Practical ',
 'Practical ',
 'Practical',
 'Practical work ',
 'Practical ',
 'Practicals',
 'Practical ',
 'Practical knowledge ',
 'practical',
 'Practical ',
 'Practicals ',
 'Practical ',
 'Practical Knowledge',
 'Practical ',
 'practicals',
 'Practical',
 'Practical ',
 'Practical ',
 'Practical ',
 'Practical ',
 'Practical',
 'good exppaletion ',
 'Practical ',
 'Practicals',
 'It was more applying rather than theory',
 'Practical ',
 'Practicals',
 'Practical class',
 'Practical',
 'It includes not only the theoretical class but also practical knowledge,that is what I likes the most.',
 'Inculeded practicals ',
 'Practical ',
 'Practical ',
 'Practical knowledge ',
 'Explanation with practical ',
 'Practical ',
 'Practical ',
 'Practicals',
 'str

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==14])

['Got new knowledge about AI .',
 'The class described the basics of ai and genAI',
 'Got an understanding about generative ai',
 'AI applications ',
 'AI tools',
 'real world problems discussed and all the background of tools discussed',
 'Learnt about more AI tools and the basics of app developing using ai',
 'Learned little about ai tools',
 'We learned more interesting about AI',
 'Creating app using Ai',
 'Introduction to the AI tools',
 'Ai and python',
 'redux toolkit',
 'About AI tools',
 'explanation ABOUT AI AND PYTHON',
 'EXPLANATION OF AI',
 'Got to now more new technologies ',
 'Important tools about AI',
 'How to utilise Ai in daily life and work',
 'Ai and Google products ',
 'AI tools',
 'an overview of ai was given',
 'Many new AI tools have been introduced',
 'Lot of tools',
 'Research paper  tools',
 'Ai tools',
 'New AI tools for education purposes ',
 'Interesting tools',
 'Introducing reserch tool',
 'curipod ai',
 'New Tools',
 'Usage of AI tools for interacting 

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==15])

['Story making ',
 'Website development',
 'developing more application in class',
 'Blog creation ',
 'blog creation ',
 'App development ',
 'Creating the first server',
 'Trying out different UI elements',
 'html',
 'Complete explanation of html ',
 'Html',
 'Html learning ',
 'html',
 'develop more applications in class',
 'css',
 'Got a good understanding about css properties and styling ',
 'css',
 'Css',
 'CSS',
 'Css',
 'website making',
 'Website demo presentation ',
 'Create front page',
 'Html',
 'Got to know more concepts of CSS ',
 'Development a front webpage ',
 'App development ',
 'Whatsapp login page',
 'Very understanding and liked making whatsapp Ui',
 'Creating page',
 'responsive webpage',
 'Instagram ui',
 'Whatsapp ',
 'Creativity',
 'learn to create whatsapp',
 'Studied how to create a whatsapp Ui ',
 'Liked  webpage design using css , javascript and html.',
 'Models ',
 ' name,gmail and three buttonsmaking an webpage ',
 'Connecting to server with openvpn.',
 

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==16])

['Made us do the code',
 'Doing programs on our own',
 'Sample programs',
 'Code',
 'About coding',
 'Got to do more coding and got better understanding',
 'Programming ',
 'Examples of JS concepts',
 'programs',
 'Coding',
 'Programming',
 'Json',
 'Json',
 'Got a good understanding about Ajax',
 'coding',
 'coding',
 'Got a good understanding about asynchronous js ',
 'Program',
 'Java ',
 'coding',
 'Coding',
 'Freedom to programming activities ',
 'programs',
 'Java hands-on ',
 'new codes',
 'Coding',
 'Coding',
 'Programming ',
 'Good flow of Java taught ',
 'Java',
 'Got a good understanding about express',
 'examples coding',
 'programs',
 'programing',
 'I really liked the quality of implying the knowledge through both lecture and practical method.This is the one main thing in many colleges and institutions lacks,which is also essential for learning coding ',
 'Explanation of the codes',
 'explanation of code',
 'explanation of the code is more good',
 'Visual Studio',
 'Colou

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==17])

['Formula',
 'Its is very convenient to learn excel',
 'Mathematical formula',
 'Formulas',
 'Macro',
 'Data modelling',
 'dtabase connection',
 'queries',
 'Doing queries ',
 'Writing function',
 'Helpful for clarifying queries ',
 '3rd party packages instalation and use',
 'Data Frame ',
 'Details about Oop',
 'data set',
 'Dataset handling is quite interested in today s session',
 'Why was the highest eigen vector points exactly towards the spread of points ',
 'Sql exercise ',
 'More knowledge in excel',
 'Confusion Matrix',
 'Could understand more about sql functions',
 'Sql functions ',
 'clustering kmeans',
 'Thank you for quoting each function with example',
 'data structures',
 'Understanding of the query ',
 'Subqueries part understanding in detail',
 'Function',
 'Handling real life data',
 'Procedure create function',
 'Procedures and function were good',
 'Excel work by using formulae',
 'Excel ',
 'Able to analyse ',
 'Network analysis',
 'R programming ',
 'Learning abou

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==18])

['Helping',
 'Helping mentality',
 'Doing together ',
 'group activities',
 'Always make sure that all are listening.',
 'Group Activity',
 'Always make sure that all are listening.',
 'Group assignments',
 'Collaborative working',
 'Always make sure that all are listening.',
 'Group Activity',
 'Team work',
 'Co operative ',
 'Group Activities',
 'Helped',
 'Group working ',
 'ensuring that everyone is done with the work',
 'Working together',
 'Helping mentality ',
 'Helping mentality',
 'Breakout activity so that we could discuss among peers.',
 'Group study ',
 'Group work',
 'Group work',
 'group activity',
 'Group activity ',
 'Group work',
 'group activity',
 'GROUP ACTIVITY',
 'the acitivity that was assigned to different teams',
 'group activity',
 'Agile principles ',
 'Group activity ',
 'Group work',
 'The team',
 'Group Project making ',
 'Clay and group activities ',
 'Group project',
 'Group study',
 'Helping the unknown',
 'Everyone come forward to help each other',
 'H

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==19])

['Engaging Class',
 'Engaging class',
 'Interest',
 'Interctive',
 'Intersting',
 'Intresting ',
 'Intresting ',
 'intresting',
 'intresting class',
 'interested class',
 'Intresting ',
 'Very intresting',
 'todays class is intresting',
 'Interacive and Good sessi',
 'Intresting ',
 'Intresting ',
 'Engaging',
 'Intresting ',
 'Internation',
 'Very intersting class',
 'Intervel ',
 'INTEDRACTION',
 'intresting class ',
 'intresting class',
 'intresting',
 'intersting',
 'intresting',
 'intresting',
 'intresting class',
 'intresting',
 'intresting',
 'intresting class',
 'Interative class',
 'intresting class',
 'intresting class',
 'Engaging ',
 'Engaging ',
 'Engaging',
 'Engaging',
 'Intersting ',
 'Engaging ',
 'Intersting ',
 'Intersting ',
 'Engaging ',
 'Intercative class',
 'Engaging class',
 'Engaging ',
 'intervel',
 'Intresting ',
 'Interation ',
 'More attractive class',
 'Engaging',
 'Interation',
 'Its Intresting',
 'Engaging ',
 'Its intresting',
 'Imformative',
 'Intrest

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==20])

['He has great patience.',
 'Patience',
 'Patience',
 'took time',
 'Thumps up',
 'Timely tasks',
 'patience',
 'time saving ',
 'time saving',
 'Patience ',
 'Patience, clears all doubts',
 'The pace of the instructor it was not too fast but not too slow either. That is good ',
 'Pace',
 'It was correctly paced',
 'Patience ',
 'Good pace',
 'very patient',
 'Engaging , Great Pace ',
 'understandable pace/language',
 'covered in short time',
 'Time management',
 'Time provided for practicing the learned sections and code',
 'Providing effective time for practicing code that are learned recently',
 'patience of the instructure',
 'understandable,patience',
 'Time management',
 'Time Management',
 'Time Management',
 'Slowpace',
 'Pace',
 'Good pace',
 'Gives time',
 'Slowpace',
 'Time management ',
 'Helpful and excellent time management',
 'Pace is apt.',
 'Pace and content was good',
 'Time giving ',
 'Time giving ',
 'proper utilization of time',
 'Punctuality ',
 'Simple to underst

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==21])

['Arrays',
 'Learning python',
 'Studying about the array topic in python',
 'basic pyton',
 'Python programming ',
 'Python Program ',
 'python window',
 'List',
 'list creation',
 'pandas',
 'Python Learning',
 'List',
 'Python learning',
 'teacher will ensure  students get an idea about python and do themself',
 'Python programming codes section',
 'Gained knowledge about python',
 'Python program application in data analysis ',
 'list',
 'list',
 'list',
 'list',
 'numpy',
 'numpy',
 'numpy',
 'numpy',
 'pandas',
 'pandas',
 'pandas',
 'Pandas',
 'pandas',
 'Discus about each and every points in python and ai ml',
 'Classes about python ',
 'It was very easy to understand the concepts of python',
 'simple and easy way to learn python,understandable',
 'The instructor explained the basics of python very well',
 'The instructer explained the basics of python very clearly',
 'pandas and numpy',
 'pandas',
 'Understood what is slicing in python',
 'Array ',
 'Array section',
 'Could re

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==22])

['interesting',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting',
 'interesting',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'interesting',
 'Interesting ',
 'Interesting',
 'Interesting ',
 'Interesting ',
 'Interesting',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting',
 'interesting',
 'interesting class',
 'interesting class',
 'Interesting class',
 'Interesting class',
 'interesting',
 'Interesting',
 'interesting',
 'Interesting',
 'Interesting ',
 'Interesting ',
 'Interesting',
 'Interesting',
 'funny',
 'funny',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting class',
 'Interesting ',
 'interesting ',
 'Interesting class',
 'Interesting class',
 'Interesting ',
 'Interesting ',
 'Interesting ',
 'Interesting class',
 'I

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==24])

['Calm ,interactive,perfect classes',
 'Good classes',
 'Good classes',
 'Classes are easily understandable',
 'Good classes',
 'Good classes',
 'Efficient classes',
 'Friendly interactive classes ',
 'Friendly interactive classes ',
 'I liked the all classes ',
 'Good classes',
 'Good classes',
 'Classes',
 'Classes',
 'The syllabus was well packed',
 'pratical classes',
 'the classes are very helful',
 'Efficient classes',
 'The syllabus was well packed',
 'Classes',
 'Good classes',
 'Good classes',
 'Was able to understand a bit more than the classes before.',
 'Classes',
 'The syllabus was well packed',
 'Good classes',
 'Good classses',
 'Good classes',
 'All classes is always nice ',
 'well defined classes',
 'Good classes',
 'understanding classes',
 'Good classes',
 'understandable and simplified classes',
 'good and understnding classes',
 'good classes',
 'interesting classes',
 'Good classes',
 'Understanding classes',
 'Understanding classes',
 'classes were interactive',


In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==25])

['Class was good',
 'It was good class',
 'Class was really good ',
 'Class was awesome',
 'It was good',
 'He is good',
 'Case study ',
 'it was good',
 'Class was excellent ',
 'Casestudy',
 'Case study',
 'case study',
 'case study',
 'casestudy',
 'Class was really good',
 'It was good',
 'Way of explaining was best ',
 'Class was really good ',
 'Case Study',
 'Class was good',
 'it was good',
 'Ladduu was awesome',
 'Class was good ',
 'Was nice class',
 'Class was good',
 'case study',
 'Class was good',
 'The class was great',
 'Class was good',
 'Class was good ',
 'Class was good',
 'Class was good',
 'Really good',
 'Last case study session',
 'Class are good',
 'Was good',
 'Class is good ',
 'Class is good',
 'It was a nice class ',
 'It was good ',
 'It was good',
 'It was good ',
 'It was good ',
 'It was good',
 'Logisim and class was good',
 'The class was so existing ',
 'It was good q',
 'Case study ',
 'Class was good.',
 'Use case',
 'Very attentive class',
 'It wa

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==26])

['Visualization ',
 'Visualizations',
 'Visualisation ',
 'Dax',
 'Exploring visualizations',
 'Image reading',
 'Image assets',
 'Just the plotting',
 'graph',
 'graph',
 'Zenmap',
 'Photoshop editing ',
 'Adobe Photoshop',
 'Class was intresting...plotting graphs and all will be helpful in future',
 'The graph and tabulations',
 'Visualization ',
 'Creating chart',
 'Chart creation ',
 'Chart',
 'Chart',
 'Chart making ',
 'Satellite tracking ',
 'Satellite image ',
 'satellite image',
 'Graph ploting',
 'STUDIED ABOUT YOLO OBJECT DETECTION ',
 'Render forest ',
 'Exel graph presentations ',
 'Charts',
 'Learning about computer vision ',
 'Computer vision demo',
 'Excel graph',
 'Image grayscaling',
 'Graph',
 'Graph',
 'Graphs',
 'Visualization practicals',
 'Topics abt visualisation',
 'Tracking air lines',
 'Flipping image',
 'Computer Vision',
 'Computer vision ',
 'More imaging programs discussion',
 'Image processing ',
 'Understood the concepts of computer vision clearly',
 'P

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==27])

['Assignment ',
 'Doing assignment ',
 'Assignment ',
 'Assignment ',
 'Assignments ',
 'Assignment ',
 'Assignment ',
 'Assignments ',
 'Assignment ',
 'Task',
 'Tasks',
 'Accountability',
 'Giving tasks',
 'Task',
 'Assignment time ',
 'task',
 'examples and tasks',
 'live tasks',
 'Giving tasks to complete',
 'Task',
 'Tasks',
 'The assignment ',
 'Assignment',
 'Researcheabbit',
 'Teaching through tasks',
 'Assignments ',
 'Assignment complition ',
 'Last assignment',
 'Last assignment ',
 'Assignment',
 'Giving task and very excited to do it.',
 'A/c',
 'Task',
 'Tasks',
 'assignment',
 'ASSIGNMENT',
 'Task',
 'The tasks they give us was good. ',
 'Task ',
 'Homework',
 'Giving more examples and tasks',
 'Task',
 'Intersting tasks',
 'assignment section',
 'Task given',
 'Crud operation ',
 'Interesting tasks',
 'Interesting tasks',
 'Last task',
 'Task',
 'Tasks',
 'Assignments ',
 'Tasks',
 'assignment',
 'assignment',
 'assignment ',
 'ASSIGNMENT',
 'assignment',
 'assignment',

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==28])

['Working on testrail tool',
 'Testrail Hands-on ',
 'Load testing',
 'About Automation testing ',
 'testing',
 'test',
 'Pentesting',
 'Test case',
 'Activities based on testing ',
 'Testing ',
 'Testing sample practices',
 'Testing using coverage',
 'Testing ',
 'Test ',
 'Testing application',
 'Testing Fundamentals ',
 'Searching types of testing ',
 'Testing types ',
 'Test case document creation',
 'Found more about test cases ',
 'test case document',
 'Making a test case document ',
 'Testing case',
 'Test plan activity , dynamic and static testing ',
 'test plan activity ',
 'Automating Workflows ',
 'Get to know more about testing tools',
 'Got to know more about Test Rail ',
 'Test rail practicals ',
 'Test rail ',
 'Test Rail',
 'testrail',
 'Train Test Split',
 'Train test split',
 'Got to more know about manual testing ',
 'Test design techniques workout',
 'JMeter practical ',
 'JMeter',
 'Jmeter ',
 'Activity in JMeter',
 'NFR & J-Meter & Activity & Quiz',
 'Jmeter test

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==29])

['Intraction',
 'Intracation tariner',
 'sections are very interactive',
 'sections are very interactive',
 'Comprehensive.',
 'intraction',
 'Friendly sections',
 'Intraction',
 'All portions ',
 'sections are very interactive',
 'Starting section ',
 'Interactive sections',
 'Intraction ',
 'Intraction ',
 'Full',
 'All sections',
 'All sections ',
 'Afternoon section ',
 'Group section ',
 'No section',
 'Intraction',
 'Try out section',
 'all section',
 'Last section ',
 'All section',
 'Helping all the sections ',
 'Forenoon section was good ',
 'Interactive sections ',
 'All of it',
 'Intraction',
 'Good traning section',
 'The way the section was handled ',
 'All parts ',
 'Good section ',
 'Nice section ',
 'Intration',
 'Interactive sections ',
 'Nice section ',
 'Understanding all sections',
 'intracting',
 'Super section ',
 'Good section ',
 'Intraction',
 'Interactive sections ',
 'Comprehensive',
 'Comprehensive',
 'Good enteractive section ',
 'All the sections are amazi

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==31])

['Cirtificate achieve ',
 'Google workspace. ',
 'Learn somethings about the apps which i thought i knew well. It was interestimg',
 'Google apps',
 'Google apps',
 'Form creation',
 'Shake app',
 'Shake app',
 'Google ads',
 'Run ads in google ads',
 'Google ads',
 'The ease of using colab',
 'Google products',
 'Google tools',
 'Google tools',
 'Google Tool',
 'Google suit ',
 'Burp suit ',
 'Gmail features',
 'Mail signature ',
 'Learned more about google docs, drive and mail',
 'Explanation was clear about google (email,drive and form)',
 'Google form ',
 'New google settings',
 'Create Google forms',
 'Got more informations about Google Tools',
 'Google forms',
 'Got more informations about google tools',
 'New google updates',
 'Making and upload google forms',
 'Google drive',
 'Gmail ',
 'Google calender',
 'form handling',
 'Google forms ',
 'Apps',
 'Google tools',
 'Google workspace',
 'Mail.merge',
 'G mail options ',
 'Email and phone validation ',
 'Form validation ',
 'F

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==32])

['Cyber security ',
 'Various field of Cyber security and opportunity',
 'Cybersecurity roles ',
 'About social media and password ',
 'Finding ip address ',
 'Basics of hacking',
 'Password cracking ',
 'Cracking',
 'Hacking',
 'Hacking',
 'Have a clear idea of Basics of cloud computing',
 'Types of Cloud Computing',
 'IP address ',
 'Ngrok crack method',
 'Hacking',
 'Authentication and authorisation explaining ',
 'Hacking ',
 'Hacking ',
 'Hacking',
 'After noon section - Try hacking ',
 'Taught more about the cloud and networks, Different types of addresses,vpc',
 'Routing',
 'Load balancer ',
 'Mern connectivity ',
 'Routing ',
 'Explained the detailed concepts about the basics of Ethical hacking ',
 'hand on practical experience and also the demonstration of most of the prominent attacks.',
 'Aws',
 'Routing',
 'Vpc',
 'Learning with experiencing on how to create vpc and related aspects ',
 'Web cam hacking ',
 'Try hackme',
 'Try hack me',
 'Dark web',
 'Dark Web',
 'Attacking 

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==33])

[' Very useful to understand flutter',
 'React is great.',
 'recap',
 'Statefull widget',
 'Was able to know more about the flutter.',
 'Stateful widget',
 'React',
 'React',
 'React ',
 'Axios methods',
 'React',
 'React',
 'Redux',
 'Redux ',
 'Learned basics of react within a small time frame ',
 'React basis ',
 'React',
 'React',
 'React',
 'React ',
 'React',
 'React ',
 'React JS',
 'Learning flutter',
 'Flutter',
 'React js snippets ',
 'New areas of react ',
 'React',
 'Creating reactapp using MERN',
 'Helpful to know about react',
 'Better understanding of React',
 'Able to easily understand react concepts while applying it in real time',
 'The new topics about React and to create projects related to it',
 'Study more about widget ',
 'React Testing',
 'The way of teaching and the resourses to learn flutter and dart. ',
 'React',
 'React explanation and implementation ',
 'Easy to connect because of the html terms used in the react ',
 'Getting to know how flutter works',
 'R

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==34])

['Very approachable',
 'Approachable',
 'easy explanation',
 'Simple explanation ',
 'Approachable faculty',
 'Underatanding',
 'Very understanble class',
 'Easy explanation ',
 'understadable',
 'undrstandable',
 'understantable',
 'understating',
 'Approachable ',
 'Approachable',
 'everything is understanable',
 'Understable. ',
 'understnding',
 'understantable',
 'understantable',
 'very understatable',
 'understantable',
 'undersstandable',
 'clearly understable',
 'understable',
 'easy to understan',
 'understant',
 'Easily undetstandable',
 'undersatanding well',
 'Simple explanation ',
 'Easy explanation ',
 'Approachable ',
 'Undertandable',
 'Intractable',
 'Understable',
 'Easy explanation ',
 'easy to undestand',
 'Easy explanation',
 'Underatanding class',
 'Friendly, understating ',
 'it is very undersanding',
 'Simple explanation ',
 'Understantable',
 'Simple explanation ',
 'under stantable',
 'Simple explanation',
 'understable',
 'understantable',
 'understadbility 

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==35])

['Pivot Tables',
 'Pivot tables',
 'Pivot Table and Lookup',
 'Pivot Table and Data Validation ',
 'Pivot table ',
 'Tableau ',
 'Pivot table',
 'Making pivot table ',
 'Pivot table',
 'Pivot table',
 'Pivot table',
 'Pivot table',
 'Pivot table',
 'Pivot table making & graph making ',
 'Pivot table',
 'Pivot table making &graph making',
 'Pivot session ',
 'Pivot ',
 'Pivot table ',
 'Pivot table',
 'Explanation of pivot table',
 'Pivot table',
 'Designing the table',
 'Pivot table',
 'pvot table',
 'Tableau public/ Google colab/ etc',
 'Tableau',
 'pivot table',
 'Tableau ',
 'Tableau is an expensive tool',
 'Tableau',
 'Pivot table',
 'Pivot Table',
 'pivot chart.',
 'Pivot table in chart making process ',
 'Tableau create.',
 'join table',
 'Pivot table',
 'Pivot Table',
 'AI tools, pivot table and their uses',
 'Pivot table Study using with AI',
 'Editing with table',
 'Table creation',
 'Pivot table and the easy equations ',
 'Pivot table session ',
 'Pvot table']

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==36])

['Jira',
 'Get understanding of jira and github',
 'Dedicated ',
 'Jira',
 'More knowledge to know about jira',
 'Jira',
 'dedication',
 'dedication',
 'Dedication ',
 'Dedication',
 'Dedication ',
 'CRUD',
 'Dedicated ',
 'Dedicated ',
 'Jira',
 'Using jera',
 'Jira',
 'Discipline ',
 'qee',
 'Sir dedication',
 'Dedication',
 'Dedication ',
 'Dedication ',
 'Jira',
 'Learning about jira',
 'Got to learn about jira',
 'Jira',
 'jira',
 'Jira',
 'Jira project ',
 'JIRA INSTALLATION, SDLC AND SCRUM ',
 'ABOUT THE JIRA SOFTWARE',
 'Jira briefly ',
 'Jira',
 'Jira ',
 'Jira signing process ',
 'Jira atlassian',
 'Jira',
 'Jira signing process ',
 'Jira',
 'Quiize ',
 'jira practice ',
 'Better understanding of jira',
 'Jiera',
 'Usage of Jira']

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==37])

['Doubt clearance ',
 'Doubt clearance',
 'doubt clearance',
 'Doubt clearance',
 'Doubt clearance',
 'doubt clearance',
 'Doubt clearance',
 'doubt clearance',
 'doubt clearance',
 'Doubt clearance',
 'doubt clearance',
 'doubt clearance',
 'Doubt clearance ',
 'Doubt clearance ',
 'Doubt clearance',
 'Doubt clearance',
 'Doubt clearance ',
 'Doubt clearance',
 'Doubt clearance',
 'Doubt clearance ',
 'Doubt clearance ',
 'Doubt clearance',
 'clearance',
 'doubt clearance',
 'Doubt clearance.',
 'Doubt clearance ',
 'Dought clearance ',
 'Doubt clearance ',
 'Doubt clearance',
 'Doubt Clearances ',
 'Doubt clearance ',
 'Doubt clearance ',
 'Doubt clearance ',
 'Doubt clearance ',
 'Doubt clearance ',
 'Doubt clearance ',
 'Doubt clearance',
 'Waited for all, easy doubt clearances ',
 'Doubt clearance ',
 'Doubt clearance',
 'Doubt clearance ',
 'Doubt clearance ',
 'Doubt clearance ',
 'Good doubt clearance']

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==38])

['About gamma app which gives ppt easily',
 'Gamma app',
 'AI for media',
 'Video creating ',
 'Different ways to make the ppt perfect',
 'Tools for preparing ppt and for research',
 'Learn new AI tools which will help to create PPT .',
 'Ai for video creation ',
 'Converting the text into a motion al video ',
 'PPT sessions',
 'Ppt presentation ',
 'Gamma app for PP',
 'AI ppt tools',
 'GAMMA.APP',
 'Ppt and project report preparation inAI',
 'Application of Gamma AI for making ppt slides',
 'Gamma for PPT making',
 'Creating ppt using AI tools',
 'Video and pptx creation ',
 'CREATION OF PROJECTS ,PPT VIDEO AND AUDIOS USING APPS',
 'Video creation ',
 'Generating ppt using prompt that also generated through ai itself',
 'PPT PREPARATION USED ON AI TOOLS',
 'AI tools for ppt presentation ',
 ' Video creation Using AI tools',
 'text prompt convert to video using notebooklm',
 'Ppt, Video and audio making using AI.',
 'Use of ai tools for the preparation of ppt, reports, letters etc',
 

In [ ]:
list(df_q10['REMARKS'][df_q10['topic']==-1])

['gave fresh perspective regarding industry readiness',
 'Spend more time for us to grabing the content',
 'Very knowledgeable ',
 'Informative, engaging',
 'Simplicity in explanation ',
 'Repeating more and more ',
 'Clear instructions and demonstrations ',
 'Hepling',
 'It was good , fun, valuable and also very  helps in future ',
 'Interactive teaching method of the instructors ',
 'Its a Good Session ',
 'Real life experience told was liked ',
 'Very helpful ',
 'Every minute things are discussed ',
 'Relevant to the course . But not able to relate or apply ',
 'He was nicely done his section . Avidannm evidannm oke koree kariyangal manasilaii',
 'The topics are very interesting ',
 'Delivery of content is clear and gave time to apply code and debug the code',
 'Clear explanation ',
 'Simple and effective.',
 'The class',
 'It was a nice interactive session',
 'Goodd',
 'It was well clear',
 'Delivery of the content was very clear',
 'The faculty repeats the steps.',
 'Remya miss i

loading model and name the topics

In [ ]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

embedding_model = SentenceTransformer("all-mpnet-base-v2")

topic_model = BERTopic.load(
    "bertopic_q10_model_min",
    embedding_model=embedding_model
)


EOFError: 

In [ ]:
topic_info = topic_model.get_topic_info()
topic_info.head(40)


,Topic,Count,Name,Representation,Representative_Docs
0,-1,2959,-1_clear_content_learning_delivery,"[clear, content, learning, delivery, interesti...","[content delivery, content delivery, content d..."
1,0,1742,0_teaching_way teaching_way_students,"[teaching, way teaching, way, students, instru...","[way of teaching, way of teaching, the way tea..."
2,1,1728,1_hands_session_training_practical session,"[hands, session, training, practical session, ...","[hands on training, hands on training, hands o..."
3,2,1653,2_interactive_interaction_interactive session_...,"[interactive, interaction, interactive session...","[interactive session, interactive session, int..."
4,3,1519,3_explanation_easy understand_understand_easy,"[explanation, easy understand, understand, eas...","[easy to understand, easy to understand, easy ..."
5,4,645,4_presentation_lecture_way_style,"[presentation, lecture, way, style, simple, sk...","[ presentation, presentation, presentation]"
6,5,478,5_communication_friendly_patient_approach,"[communication, friendly, patient, approach, i...","[communication, communication, friendly inter..."
7,6,409,6_topics_topic_new_new things,"[topics, topic, new, new things, things, learn...","[topics, recap of older topics, new topic for me]"
8,7,407,7_comfortable_active_free_amazing,"[comfortable, active, free, amazing, liked, ca...","[comfortable, feels free and comfortable, comf..."
9,8,393,8_understandable___,"[understandable, , , , , , , , , ]","[understandable, understandable, understandable]"


In [ ]:
topic_keywords = {}

for topic_id in topic_info.Topic:
    if topic_id != -1:
        words = topic_model.get_topic(topic_id)
        topic_keywords[topic_id] = [word for word, _ in words]


In [ ]:
topic_keywords

{0: ['teaching',
  'way teaching',
  'way',
  'students',
  'instructor',
  'method',
  'style',
  'taught',
  'interaction',
  'skill'],
 1: ['hands',
  'session',
  'training',
  'practical session',
  'practical',
  'sessions',
  'practice',
  'experience',
  'hands experience',
  'practice session'],
 2: ['interactive',
  'interaction',
  'interactive session',
  'session',
  'sessions',
  'demo',
  'live',
  'interacting',
  'interactions',
  'mode'],
 3: ['explanation',
  'easy understand',
  'understand',
  'easy',
  'understanding',
  'explained',
  'easily',
  'clearly',
  'understandable',
  'explanations'],
 4: ['presentation',
  'lecture',
  'way',
  'style',
  'simple',
  'skill',
  'support',
  'clear',
  'crystal',
  'crystal clear'],
 5: ['communication',
  'friendly',
  'patient',
  'approach',
  'interacting',
  'skills',
  'skill',
  'friendly interactive',
  'proper',
  'approachable'],
 6: ['topics',
  'topic',
  'new',
  'new things',
  'things',
  'learned',
  'l

In [ ]:
representative_docs = topic_model.get_representative_docs()

In [ ]:
review_rows = []

for topic_id, keywords in topic_keywords.items():
    review_rows.append({
        "Topic": topic_id,
        "Keywords": ", ".join(keywords[:10]),
        "Example_Comments": " | ".join(representative_docs[topic_id][:3])
    })

topic_review_df = pd.DataFrame(review_rows)
topic_review_df


,Topic,Keywords,Example_Comments
0,0,"teaching, way teaching, way, students, instruc...",way of teaching | way of teaching | the way te...
1,1,"hands, session, training, practical session, p...",hands on training | hands on training | hands ...
2,2,"interactive, interaction, interactive session,...",interactive session | interactive session | in...
3,3,"explanation, easy understand, understand, easy...",easy to understand | easy to understand | easy...
4,4,"presentation, lecture, way, style, simple, ski...",presentation | presentation | presentation
5,5,"communication, friendly, patient, approach, in...",communication | communication | friendly inte...
6,6,"topics, topic, new, new things, things, learne...",topics | recap of older topics | new topic for me
7,7,"comfortable, active, free, amazing, liked, cal...",comfortable | feels free and comfortable | com...
8,8,"understandable, , , , , , , , ,",understandable | understandable | understandable
9,9,"clearing, doubt clearing, doubt, doubts, clari...",immediate doubt clearing | doubt clearing | do...


In [ ]:
topic_name_mapping = {
    0: "Teaching Style & Delivery",
    1: "Hands-On & Practical Learning Experience",
    2: "Interactive Learning Sessions",
    3: "Easy-to-Understand Explanation",
    4: "Presentation Clarity & Lecture Delivery",
    5:"Trainer Communication & Approachability",
    6:"Topic Clarity & Knowledge Gained",
    7:"Stress-Free and Supportive Environment",
    8:"Ease of Understanding",
    9:"Effective Doubt Clearing & Clarification",
    10:"Activity-Based & Game-Oriented Learning",
    11:"Quiz-Based Practice & Problem Solving",
    12:"Informative & Effective Learning Experience",
    13:"Practical-Oriented Learning Approach",
    14:"Introduction to AI Tools & Applications",
    15:"Hands-On App & Web Development",
    16:"Hands-On Coding & Programming Practice",
    17:"Data Analysis Tools & Excel-Based Learning",
    18:"Collaborative Learning & Teamwork",
    19:"Engaging & Interesting Classes",
    20:"Effective Time Management & Learning Pace",
    21:"Python Basics & Practical Programming",
    22:"Enjoyable & Fun Learning Experience",
    23:"Overall Positive Learning Experience",
    24:"Clear & Well-Structured Classes",
    25:"Case Study–Based Learning",
    26:"Tool Setup, Installation & Exploration",
    27:"Task-Based Learning & Assignments",
    28:"Software Testing Tools & Practices",
    29:"Interactive & Inclusive Session Structure",
    30:"AI-Assisted Content & Tool Creation",
    31:"Google Workspace Tools & Applications",
    32:"MERN Stack Basics & Guided Practice",
    33:"Advanced MERN Concepts & Projects",
    34:"Approachable Teaching & Clear Explanation",
    35:"Pivot Tables & Data Visualization Tools",
    36:"Visualization, Graph Plotting & Computer Vision",
    37:"Effective & Easy Doubt Clearance",
    38:"AI-Powered Content Creation (PPT, Video & Media)"
}


In [ ]:
topic_model.set_topic_labels(topic_name_mapping)

In [ ]:
topic_model.save(
    "bertopic_q10_final_named_model",
    serialization="pickle"
)

2025-12-28 18:33:15,602 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


load final modeling and apply to new feedback

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

topic_model = BERTopic.load(
    "bertopic_q10_final_named_model"
)


In [ ]:
q10_df = df_q10[df_q10["COURSE_ID"]==30]
q10_df

,COURSE_ID,FEEDBACK_ID,KO,DATE,QUESTION_ID,VALUE,STUDENT_ID,TIMESTAMP,TYPE,REMARKS,SUBOU_ID,OU_ID,OU,OUS
1004,30,69,88.0,2024-11-23,10,0,467,2024-11-23 15:40:14.653,2,Nil,6,4,ACADEMIC,ACADEMIC (SOUTH)
1007,30,69,88.0,2024-11-23,10,0,468,2024-11-23 15:40:42.607,2,Doing together,6,4,ACADEMIC,ACADEMIC (SOUTH)
1010,30,69,88.0,2024-11-23,10,0,469,2024-11-23 15:40:55.077,2,Practice sessions,6,4,ACADEMIC,ACADEMIC (SOUTH)
1013,30,69,88.0,2024-11-23,10,0,470,2024-11-23 15:41:01.403,2,Very good,6,4,ACADEMIC,ACADEMIC (SOUTH)
1016,30,69,88.0,2024-11-23,10,0,486,2024-11-23 15:45:22.643,2,Quizz,6,4,ACADEMIC,ACADEMIC (SOUTH)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16497,30,69,87.0,2025-02-24,10,0,485,2025-02-24 15:54:34.610,2,Good training,6,4,ACADEMIC,ACADEMIC (SOUTH)
16501,30,69,87.0,2025-02-24,10,0,11707,2025-02-24 15:53:12.687,2,Class was good,6,4,ACADEMIC,ACADEMIC (SOUTH)
16503,30,69,87.0,2025-02-24,10,0,471,2025-02-24 15:54:57.800,2,Doubts clearing,6,4,ACADEMIC,ACADEMIC (SOUTH)
16506,30,69,87.0,2025-02-24,10,0,11716,2025-02-24 15:55:27.953,2,Way of presentation,6,4,ACADEMIC,ACADEMIC (SOUTH)


In [ ]:
q10_df.shape

(225, 14)

In [ ]:
NON_INFORMATIVE_PATTERNS = [
    r"^nothing$",
    r"^nothing much$",
    r"^nothing in particular$",
    r"^no comment(s)?$",
    r"^null$",
    r"^n/?a$",
    r"^na$",
    r"^everything$",
    r"^everything is (good|fine|ok|alright)(.*)?$",
    r"^overall (good|fine|ok|excellent|nice)(.*)?$",
    r"^it was excellent(.*)?$",
    r"^excellent(.*)?$",
    r"^good$",
    r"^very good(.*)?$",
    r"^wonderful(.*)?$",
    r"^outstanding(.*)?$",
    r"^like everything(.*)?$",
    r"^all (thing|things)(.*)?$",
    r"^nice sir$",
    r"^good sir$",
    r"^all$",
    r"^none$",
    r"^nthg$",
    r"^nan$",
    r"^nil$",
    r"good class$",
    r"nice class$",
    r"its ok$",
    r"nothimg$",
    r"gud$",
    r"non$",
    r"nop$",
    r"yes$",
    r"nul$",
    r"nuce$",
    r"noo$",
    r"really$"
]
TEACHING_CONTEXT = [
    "class", "lecture", "teaching", "explain", "interactive",
    "boring", "interesting", "pace", "topic", "sir", "teacher", "miss"
]

GENERIC_WORDS = {
    "good", "nice", "excellent", "awesome",
    "great", "best", "perfect",
    "sir", "madam", "teacher", "faculty", "class", "miss"
}

GENERIC_PATTERN = r"\b(" + "|".join(GENERIC_WORDS) + r")\b"


In [ ]:
def is_non_informative(text: str) -> bool:
    text = text.lower().strip()
    for pattern in NON_INFORMATIVE_PATTERNS:
        if re.fullmatch(pattern, text):
            return True
    return False

def normalize_section_session(text: str) -> str:
    if "section" in text:
        for w in TEACHING_CONTEXT:
            if w in text:
                return re.sub(r"\bsection\b", "session", text)
    return text

def preprocess_for_bertopic(text: str):
    if pd.isna(text):
        return None

    text = str(text).lower().strip()

    # Drop non-informative responses
    if is_non_informative(text):
        return None

    # Remove generic praise words
    text = re.sub(GENERIC_PATTERN, " ", text)

    # Normalize section → session (context-aware)
    text = normalize_section_session(text)

    # Remove numbers & symbols
    text = re.sub(r"[^a-z\s]", " ", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # Drop very short texts
    if len(text) < 3:
        return None

    return text


In [ ]:
q10_df = q10_df[
    (q10_df["REMARKS"].notna()) &
    (q10_df["REMARKS"].astype(str).str.strip() != "")
].copy()

q10_df["raw_text"] = q10_df["REMARKS"].astype(str)
q10_df["cleaned"] = q10_df["raw_text"].apply(preprocess_for_bertopic)

q10_df = q10_df[q10_df["cleaned"].notna()]
new_feedback = q10_df["cleaned"].tolist()


In [ ]:
topics, probs = topic_model.transform(new_feedback)

output_df = pd.DataFrame({
    "Feedback": new_feedback,
    "Topic_ID": topics,
    "Confidence": probs.max(axis=1)
})


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
topic_keywords = topic_model.get_topic_info()[["Topic", "Name"]]

output_df = output_df.merge(
    topic_keywords,
    left_on="Topic_ID",
    right_on="Topic",
    how="left"
).drop(columns=["Topic"])


In [ ]:
topic_model.topic_labels_


{-1: '-1_clear_content_learning_delivery',
 0: '0_teaching_way teaching_way_students',
 1: '1_hands_session_training_practical session',
 2: '2_interactive_interaction_interactive session_session',
 3: '3_explanation_easy understand_understand_easy',
 4: '4_presentation_lecture_way_style',
 5: '5_communication_friendly_patient_approach',
 6: '6_topics_topic_new_new things',
 7: '7_comfortable_active_free_amazing',
 8: '8_understandable___',
 9: '9_clearing_doubt clearing_doubt_doubts',
 10: '10_activities_activity_based_individual',
 11: '11_quiz_questions_problems_problem',
 12: '12_informative_effective_useful_work',
 13: '13_practicals_practical_theory_approach',
 14: '14_ai_tools_ai tools_new',
 15: '15_app_development_making_creating',
 16: '16_coding_programming_code_programs',
 17: '17_excel_data_analysis_data analysis',
 18: '18_helping_team_activity_work',
 19: '19_intresting_engaging_intersting_interested',
 20: '20_time_pace_management_slow',
 21: '21_python_basics_programmi

In [ ]:
topic_name_mapping = {
    0: "Teaching Style & Delivery",
    1: "Hands-On & Practical Learning Experience",
    2: "Interactive Learning Sessions",
    3: "Easy-to-Understand Explanation",
    4: "Presentation Clarity & Lecture Delivery",
    5: "Trainer Communication & Approachability",
    6: "Topic Clarity & Knowledge Gained",
    7: "Stress-Free and Supportive Environment",
    8: "Ease of Understanding",
    9: "Effective Doubt Clearing & Clarification",
    10: "Activity-Based & Game-Oriented Learning",
    11: "Quiz-Based Practice & Problem Solving",
    12: "Informative & Effective Learning Experience",
    13: "Practical-Oriented Learning Approach",
    14: "Introduction to AI Tools & Applications",
    15: "Hands-On App & Web Development",
    16: "Hands-On Coding & Programming Practice",
    17: "Data Analysis Tools & Excel-Based Learning",
    18: "Collaborative Learning & Teamwork",
    19: "Engaging & Interesting Classes",
    20: "Effective Time Management & Learning Pace",
    21: "Python Basics & Practical Programming",
    22: "Enjoyable & Fun Learning Experience",
    23: "Overall Positive Learning Experience",
    24: "Clear & Well-Structured Classes",
    25: "Case Study–Based Learning",
    26: "Tool Setup, Installation & Exploration",
    27: "Task-Based Learning & Assignments",
    28: "Software Testing Tools & Practices",
    29: "Interactive & Inclusive Session Structure",
    30: "AI-Assisted Content & Tool Creation",
    31: "Google Workspace Tools & Applications",
    32: "MERN Stack Basics & Guided Practice",
    33: "Advanced MERN Concepts & Projects",
    34: "Approachable Teaching & Clear Explanation",
    35: "Pivot Tables & Data Visualization Tools",
    36: "Visualization, Graph Plotting & CV",
    37: "Easy Doubt Clearance",
    38: "AI-Powered Content Creation"
}
topic_model.set_topic_labels(topic_name_mapping)
topic_model.topic_labels_[1]

'1_hands_session_training_practical session'

In [ ]:
topic_label_map = topic_model.topic_labels_

output_df["Topic_name"] = output_df["Topic_ID"].map(topic_name_mapping)
output_df["Topic_name"] = output_df["Topic_name"].fillna("outlier / Emerging Topic")


In [ ]:
normal_feedback = output_df[output_df["Topic_ID"] != -1]
outlier_feedback = output_df[output_df["Topic_ID"] == -1]


In [ ]:
"""topic_summary = (
    normal_feedback
    .groupby(["Topic_ID", "Name"])
    .size()
    .reset_index(name="Frequency")
    .sort_values("Frequency", ascending=False)
)
"""

In [ ]:
topic_summary = (
    normal_feedback
    .groupby(["Topic_ID", "Topic_name", "Name"])
    .size()
    .reset_index(name="Frequency")
)

topic_summary["Percentage"] = (
    topic_summary["Frequency"] / topic_summary["Frequency"].sum()
) * 100

topic_summary = topic_summary.sort_values("Percentage", ascending=False)


In [ ]:
topic_summary

,Topic_ID,Topic_name,Name,Frequency,Percentage
0,0,Teaching Style & Delivery,0_teaching_way teaching_way_students,47,29.936306
1,1,Hands-On & Practical Learning Experience,1_hands_session_training_practical session,23,14.649682
2,2,Interactive Learning Sessions,2_interactive_interaction_interactive session_...,22,14.012739
3,3,Easy-to-Understand Explanation,3_explanation_easy understand_understand_easy,11,7.006369
11,11,Quiz-Based Practice & Problem Solving,11_quiz_questions_problems_problem,8,5.095541
6,6,Topic Clarity & Knowledge Gained,6_topics_topic_new_new things,7,4.458599
4,4,Presentation Clarity & Lecture Delivery,4_presentation_lecture_way_style,6,3.821656
20,30,AI-Assisted Content & Tool Creation,30_creation_creating_excel_using,5,3.184713
5,5,Trainer Communication & Approachability,5_communication_friendly_patient_approach,4,2.547771
15,18,Collaborative Learning & Teamwork,18_helping_team_activity_work,4,2.547771


In [ ]:
outlier_ratio = len(outliers) / len(raw_feedback)

if outlier_ratio >= 0.15 and len(outliers) >= 5:
    print("🔍 New patterns detected in outliers")
    outlier_texts = outlier_feedback["Feedback"].tolist()

    outlier_model = BERTopic(
        embedding_model=topic_model.embedding_model,
        min_topic_size=5,
        verbose=True
    )

    outlier_topics, _ = outlier_model.fit_transform(outlier_texts)

    new_topic_info = outlier_model.get_topic_info()
    new_topic_info = new_topic_info[new_topic_info.Topic != -1]

    representative_docs = outlier_model.get_representative_docs()


2025-12-29 17:22:59,500 - BERTopic - Embedding - Transforming documents to embeddings.


🔍 New patterns detected in outliers


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

2025-12-29 17:23:01,115 - BERTopic - Embedding - Completed ✓
2025-12-29 17:23:01,116 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-29 17:23:11,469 - BERTopic - Dimensionality - Completed ✓
2025-12-29 17:23:11,471 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-29 17:23:11,500 - BERTopic - Cluster - Completed ✓
2025-12-29 17:23:11,507 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-29 17:23:11,554 - BERTopic - Representation - Completed ✓


In [ ]:
representative_docs

{0: ['clear explanation',
  'explains from the basics even for a beginner can also follow',
  'the session of this topic which is completely different one'],
 1: ['quizizz', 'quizizz', 'quizizz']}

In [ ]:
new_topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,0,28,0_the_explanation_interactive_of,"[the, explanation, interactive, of, clear, con...","[clear explanation, explains from the basics e..."
1,1,6,1_quizizz_quizz_quizzez_,"[quizizz, quizz, quizzez, , , , , , , ]","[quizizz, quizizz, quizizz]"


In [ ]:
output_df.to_csv("assigned_feedback.csv", index=False)
topic_summary.to_csv("teaching_methods_summary.csv", index=False)
new_topic_info.to_csv("emerging_teaching_methods.csv", index=False)


In [ ]:
new_topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,0,28,0_the_explanation_interactive_of,"[the, explanation, interactive, of, clear, con...","[clear explanation, explains from the basics e..."
1,1,6,1_quizizz_quizz_quizzez_,"[quizizz, quizz, quizzez, , , , , , , ]","[quizizz, quizizz, quizizz]"


versions

In [ ]:
import bertopic, torch, sklearn, umap, hdbscan
import sentence_transformers

print("bertopic:", bertopic.__version__)
print("torch:", torch.__version__)
print("sklearn:", sklearn.__version__)
print("umap:", umap.__version__)
# print("hdbscan:", hdbscan.__version__) # Removed this line
print("sentence-transformers:", sentence_transformers.__version__)


bertopic: 0.17.4
torch: 2.9.0+cpu
sklearn: 1.6.1
umap: 0.5.9.post2
sentence-transformers: 5.2.0


In [ ]:
from importlib.metadata import version
print("hdbscan:", version("hdbscan"))

hdbscan: 0.8.41


In [ ]:
import sys
print(sys.version)

3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
